In [1]:
import pandas as pd
from io import StringIO
import requests
import mysql.connector
from mysql.connector import Error

In [2]:
# ARK Invest API base url
base_url = 'https://ark-funds.com/wp-content/uploads/funds-etf-csv/'

# ARK Invest ETF names
funds = ['ARK_INNOVATION_ETF_ARKK_HOLDINGS',
'ARK_NEXT_GENERATION_INTERNET_ETF_ARKW_HOLDINGS',
'ARK_FINTECH_INNOVATION_ETF_ARKF_HOLDINGS',
'THE_3D_PRINTING_ETF_PRNT_HOLDINGS',
'ARK_GENOMIC_REVOLUTION_ETF_ARKG_HOLDINGS',
'ARK_AUTONOMOUS_TECH._&_ROBOTICS_ETF_ARKQ_HOLDINGS',
'ARK_SPACE_EXPLORATION_&_INNOVATION_ETF_ARKX_HOLDINGS',
'ARK_ISRAEL_INNOVATIVE_TECHNOLOGY_ETF_IZRL_HOLDINGS']

# Request headers
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

In [3]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df1 = df.copy()
    df1.drop(df.index[-1], inplace=True) # drop last row
    df1.fillna(0, inplace=True) # fill NaN with 0
    df1.rename(columns={'market value ($)': 'market_value', 'weight (%)': 'weight'}, inplace=True) # rename columns
    df1['date'] = pd.to_datetime(df1['date'], format="%m/%d/%Y") # Convert the "date" column to datetime
    df1['market_value'] = df1['market_value'].str.replace(r'[^\d.]', '', regex=True).astype(float) # remove $ and , and convert to float
    df1['shares'] = df1['shares'].str.replace(r'[^\d.]', '', regex=True).astype(int) # remove , and convert to float
    df1['weight'] = df1['weight'].str.replace(r'[^\d.]', '', regex=True).astype(float) / 100 # remove % and convert to unitary float
    return df1

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

def insert_data_into_db(data_frame):
    try:
        db_config = {
            "host": "ark-digest.crubyqmjsrku.eu-west-3.rds.amazonaws.com",
            "user": os.getenv('DB_USER'),
            "password": os.getenv('DB_PASS'),
            "database": "ark_digest",
        }

        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor()

        table_name = 'holdings' 

        for index, row in data_frame.iterrows():
            try:
                insert_query = f"""
                    INSERT INTO {table_name} (date, fund, company, ticker, cusip, shares, market_value, weight)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """

                data_tuple = (
                    row['date'], row['fund'], row['company'], row['ticker'],
                    row['cusip'], row['shares'], row['market_value'], row['weight']
                )

                cursor.execute(insert_query, data_tuple)
                connection.commit()

            except Error as e:
                print(f"Error inserting row {index}: {e}")
                connection.rollback()  # Rollback the transaction for the current row

        print("Data insertion completed!")

    except Error as e:
        print("Error connecting to the database:", e)

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()



In [5]:
for fund in funds:

    url = f'{base_url}{fund}.csv'

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        csv_content = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(csv_content)) 
        df = clean_data(df)
        insert_data_into_db(df)
    else:
        print(f"Failed to download the CSV file. Status code: {response.status_code}")

Data insertion completed!
Data insertion completed!
Data insertion completed!
Data insertion completed!
Data insertion completed!
Data insertion completed!
Data insertion completed!
Data insertion completed!
